# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum += sqrt_value
print(f'Sum with limit={limit}:', sum)


Sum with limit=5: 8.382332347441762


In [9]:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [10]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [12]:
!pip install dlt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.2/540.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.9 MB/s eta 0:00:00


In [29]:
import dlt
import duckdb

pipeline = dlt.pipeline(destination='duckdb', dataset_name='persons_dataset')

info = pipeline.run(people_1(), table_name="persons_table", write_disposition="replace")
print(info)

conn =  duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

table_df = conn.sql("SELECT * FROM persons_table").df()
print(table_df)

print('Sum of ages:', table_df.age.sum())

Pipeline dlt_colab_kernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707589440.9754689 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ persons_table       │
└─────────────────────┘

   id      name  age    city        _dlt_load_id         _dlt_id occupation
0   1  Person_1   26  City_A  1707589440.9754689  IBwawAOWa3rd5g       None
1   2  Person_2   27  City_A  1707589440.9754689  v6U7upLA31xT8w       None
2   3  Person_3   28  City_A  1707589440.9754689  M8HxoFhg8gHybw       None
3   4  Person_4   29  City_A  1707589440.9754689  NfQcuovbf0V8tA       None
4   5  Person_5   30  City_A  1707589440.9754689  voTthzTA6u8nAg       None
Sum of ages: 140


In [30]:
info = pipeline.run(people_2(), table_name="persons_table", write_disposition="append")
print(info)

table_df = conn.sql("SELECT * FROM persons_table").df()
print(table_df)

print('Sum of ages:', table_df.age.sum())


Pipeline dlt_colab_kernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707589458.4490492 is LOADED and contains no failed jobs
    id      name  age    city        _dlt_load_id         _dlt_id occupation
0    1  Person_1   26  City_A  1707589440.9754689  IBwawAOWa3rd5g       None
1    2  Person_2   27  City_A  1707589440.9754689  v6U7upLA31xT8w       None
2    3  Person_3   28  City_A  1707589440.9754689  M8HxoFhg8gHybw       None
3    4  Person_4   29  City_A  1707589440.9754689  NfQcuovbf0V8tA       None
4    5  Person_5   30  City_A  1707589440.9754689  voTthzTA6u8nAg       None
5    3  Person_3   33  City_B  1707589458.4490492  GfRcX577cDW+yw      Job_3
6    4  Person_4   34  City_B  1707589458.4490492  Uonte6uzLovuxQ      Job_4
7    5  Person_5   35  City_B  1707589458.4490492  1ciO

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [37]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [38]:
import dlt
import duckdb

pipeline = dlt.pipeline(destination='duckdb', dataset_name='persons_dataset')

info = pipeline.run(people_1(), table_name="persons_table", write_disposition="replace", primary_key="id")
print(info)

info = pipeline.run(people_2(), table_name="persons_table", write_disposition="merge", primary_key="id")
print(info)

conn =  duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

table_df = conn.sql("SELECT * FROM persons_table").df()
print(table_df)

print('Sum of ages:', table_df.age.sum())

Pipeline dlt_colab_kernel_launcher load step completed in 0.28 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707589833.0864842 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707589834.0238261 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ persons_table       │
└─────────────────────┘

   id      name  age    city        _dlt_load_id         _dlt_id occupation
0   1  Person_1   26  City_A  1707589833.0864842  zM6GZo6V9GnaMA       None
1   2  Person_2   27  City_A  1707589833.0864842  c2nzthIOWnz1wA       None
2   5  Person_5   35  City_B  1707589834.0238261  j+bVBO8dasyXQQ      Job_5
3   8  Person_8   38  City_B  1707589834.0238261  D2rhcrCK1u/GOA      Job_8
4   7  Person_7   37  City_B  1707589834.0238261  rromipEovGbNpA      Job_7
5   4  Person_4   34  City_B  1707589834.0238261  l+yCGDTdUj/Gtw      Job_4
6   3  Person_3   33  City_B  1707589834.0238261  SHRsKlfLpJAAjQ      Job_3
7   6  Person_6   36  City_B  1707589834.0238261  xu43lrw2g1FfMQ      Job_6
Sum of ages: 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX